## Project: 3D Motion Planning
![Quad Image](./misc/enroute.png)

---

# Required Steps for a Passing Submission:
1. Load the 2.5D map in the colliders.csv file describing the environment.
2. Discretize the environment into a grid or graph representation.
3. Define the start and goal locations.
4. Perform a search using A* or other search algorithm.
5. Use a collinearity test or ray tracing method (like Bresenham) to remove unnecessary waypoints.
6. Return waypoints in local ECEF coordinates (format for `self.all_waypoints` is [N, E, altitude, heading], where the drone’s start location corresponds to [0, 0, 0, 0].
7. Write it up.
8. Congratulations!  Your Done!

## [Rubric](https://review.udacity.com/#!/rubrics/1534/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  

You're reading it! Below I describe how I addressed each rubric point and where in my code each point is handled.


### Explain the Starter Code

#### 1. Explain the functionality of what's provided in `motion_planning.py` and `planning_utils.py`
These scripts contain a basic planning implementation that includes...

And here's a lovely image of my results (ok this image has nothing to do with it, but it's a nice example of how to include images in your writeup!)
![Top Down View](./misc/high_up.png)

Here's | A | Snappy | Table
--- | --- | --- | ---
1 | `highlight` | **bold** | 7.41
2 | a | b | c
3 | *italic* | text | 403
4 | 2 | 3 | abcd

#### Q1. Test that motion_planning.py is a modified version of backyard_flyer_solution.py for simple path planning. Verify that both scripts work. Then, compare them side by side and describe in words how each of the modifications implemented in motion_planning.py is functioning.

(fcnd) 88michaeldeMacBook-Air:FCND-Motion-Planning 88michael$ python motion_planning.py
Logs/TLog.txt
Logs/NavLog.txt
starting connection
arming transition
Searching for a path ...
global home [-122.39745   37.79248    0.     ], position [-1.22397493e+02  3.77924943e+01  9.30000000e-02], local position [ 1.59504807 -3.82608819 -0.09381434]
North offset = -316, east offset = -445
Local Start and Goal:  (316, 445) (326, 455)
Found a path.
Sending waypoints to simulator ...
takeoff transition
waypoint transition
target position [0, 0, 5, 0]
waypoint transition
target position [0, 1, 5, 0]
waypoint transition
target position [1, 1, 5, 0]
waypoint transition
target position [1, 2, 5, 0]
waypoint transition
target position [2, 2, 5, 0]
waypoint transition
target position [2, 3, 5, 0]
waypoint transition
target position [3, 3, 5, 0]
waypoint transition
target position [3, 4, 5, 0]
waypoint transition
target position [4, 4, 5, 0]
waypoint transition
target position [4, 5, 5, 0]
waypoint transition
target position [5, 5, 5, 0]
waypoint transition
target position [5, 6, 5, 0]
waypoint transition
target position [6, 6, 5, 0]
waypoint transition
target position [6, 7, 5, 0]
waypoint transition
target position [7, 7, 5, 0]
waypoint transition
target position [7, 8, 5, 0]
waypoint transition
target position [8, 8, 5, 0]
waypoint transition
target position [8, 9, 5, 0]
waypoint transition
target position [9, 9, 5, 0]
waypoint transition
target position [9, 10, 5, 0]
waypoint transition
target position [10, 10, 5, 0]
landing transition
disarm transition
manual transition
Closing connection ...

(fcnd) 88michaeldeMacBook-Air:FCND-Motion-Planning 88michael$ python backyard_flyer_solution.py
Logs/TLog.txt
Logs/NavLog.txt
starting connection
arming transition
takeoff transition
Setting Home
waypoint transition
target position [10.0, 0.0, 3.0]
waypoint transition
target position [10.0, 10.0, 3.0]
waypoint transition
target position [0.0, 10.0, 3.0]
waypoint transition
target position [0.0, 0.0, 3.0]
landing transition
disarm transition
manual transition
Closing connection ...


#### Answer:

motion_planning.py works almost same as backyard_flyer_solution.py in the state basic control loop for managing the drones behaviour, such as starting connection, arming transition, landing transition, disarm transition, closing connection. But there are several different basic planning implementation in the process of takeoff transition and waypoint transition to plan a suitable path.

In the process of takeoff transition:  motion_planning.py is added plan_path function to calculate path before takeoff transition, whereas backyard_flyer_solution create calculate_box function to shape flight path after takeoff transitin.
In the plan_path function of motion_planning.py, there are three keypoints to find path. Initially, it makes grid representation of 2D configuration space given obstacle data with TARGET_ALTITUDE and SAFETY_DISTANCE. It also use UTM package to convert global positon to local position(NED) and set start point as center of the map. In addition, it add searching path functions such as A* algorithm and collinearity algorithm to optimize implementation.

In the phase of waypoint transiton: motion_planning.py is not only sending waypoints to simulator, but also converting path to waypoints by adding north and east offset.

### Implementing Your Path Planning Algorithm

#### 1. Set your global home position
Here students should read the first line of the csv file, extract lat0 and lon0 as floating point values and use the self.set_home_position() method to set global home. Explain briefly how you accomplished this in your code.


And here is a lovely picture of our downtown San Francisco environment from above!
![Map of SF](./misc/map.png)



#### Answer:

The drone needs to know the local coordinate of each successive waypoint to generate a vaid path. Reading configuration space state named 'colliders.csv' file is able to accomplish setting the home location.

In [4]:
with open('colliders.csv') as f:
    home_pos_data = f.readline().split(",")
lat0 = float(home_pos_data[0].strip().split(" ")[1])
lon0 = float(home_pos_data[1].strip().split(" ")[1])
print(lat0, lon0)

37.79248 -122.39745


#### 2. Set your current local position
Here as long as you successfully determine your local position relative to global home you'll be all set. Explain briefly how you accomplished this in your code.


Meanwhile, here's a picture of me flying through the trees!
![Forest Flying](./misc/in_the_trees.png)





#### Answer:

In the step one, the quadcopter obtain the global coordinates, then it needs to identify the home position. This is achieved by the following code:

#### TODO: set home position to (lon0, lat0, 0)
self.set_home_position(lon0, lat0, 0)

#### 3. Set grid start position from local position
This is another step in adding flexibility to the start location. As long as it works you're good to go!




#### Answer:

UTM conversion library for Python and frame_utils module create a global_to_local function to calculate the NED coordinates from the global position. In the NED frame, the world is constructed as grid representation and discretized to reduce the size of the grid space so that the minimum data points build up reasonable grid. That means the relative center is the offset of north, east, down direction.

#### TODO: retrieve current global position
current_local_position=global_to_local(self.global_position, self.global_home)

grid, north_offset, east_offset = create_grid(data, TARGET_ALTITUDE, SAFETY_DISTANCE)
print("North offset = {0}, east offset = {1}".format(north_offset, east_offset))
#### Define starting point on the grid (this is just grid center)
grid_start = (int(current_local_position[0]-north_offset), int(current_local_position[1]-east_offset))
#### TODO: convert start position to current position rather than map center
local_goal = global_to_local(self.global_position, self.global_home)
grid_goal = (int(local_goal[0]-north_offset+50), int(local_goal[1]-east_offset+60))

#### 4. Set grid goal position from geodetic coords
This step is to add flexibility to the desired goal location. Should be able to choose any (lat, lon) within the map and have it rendered to a goal location on the grid.



#### Answer:

In the above step, offsets are calculated, the local goal is taken converting global latitude and global longitude. Picked up [-122.4081522, 37.7942581] as cocal goal location.

#### TODO: adapt to set goal as latitude / longitude position and convert
local_goal[0]=-122.4081522;  
local_goal[1]=37.7942581

#### 5. Modify A* to include diagonal motion (or replace A* altogether)
Minimal requirement here is to modify the code in planning_utils() to update the A* implementation to include diagonal motions on the grid that have a cost of sqrt(2), but more creative solutions are welcome. Explain the code you used to accomplish this step.



#### Answer:

A* implementation upgrade diagonal motions to action enumeration and define cost of diagonal motion to sqrt(2). The valid_actions function also extends to identify drone actions not move outside the grid or hit obstacles. 

#### TODO:
    SOUTH_EAST=(1, 1, sqrt(2));
    NORTH_EAST=(-1, 1, sqrt(2));
    SOUTH_WEST=(1, -1, sqrt(2));
    NORTH_WEST=(-1, -1, sqrt(2));
    
valid_actions:

    if x + 1 > n or y + 1 > m or grid[x + 1, y + 1] == 1:
        valid_actions.remove(Action.SOUTH_EAST)
    if x - 1 < 0 or y + 1 > m or grid[x - 1, y + 1] == 1:
        valid_actions.remove(Action.NORTH_EAST)
    if x + 1 > n or y - 1 < 0 or grid[x + 1, y - 1] ==1:
        valid_actions.remove(Action.SOUTH_WEST)
    if x - 1 < 0 or y - 1 <0 or grid[x - 1, y - 1] ==1:
        valid_actions.remove(Action.NORTH_WEST)

#### 6. Cull waypoints 
For this step you can use a collinearity test or ray tracing method like Bresenham. The idea is simply to prune your path of unnecessary waypoints. Explain the code you used to accomplish this step.

#### Answer:

Collinearity Prune: to reduce the number of waypoints, using collinearity prune method to remove series of collinear points and make efficient waypoints.

#### TODO:

def collinearity_prune(path, epsilon=1e-5):

    """
    Prune path points from `path` using collinearity.
    """
    def point(p):
        return np.array([p[0], p[1], 1.]).reshape(1, -1)
    
    def collinearity_check(p1, p2, p3):
        m=np.concatenate((p1, p2, p3), 0)
        det = np.linalg.det(m)
        return abs(det)<epsilon
    pruned_path=[p for p in path]
    i = 0
    while i < len(pruned_path) -2:
        p1=point(pruned_path[i])
        p2=point(pruned_path[i+1])
        p3=point(pruned_path[i+2])
        if collinearity_check(p1, p2, p3):
            pruned_path.remove(pruned_path[i+1])
        else:
            i+=1
    return pruned_path

### Execute the flight
#### 1. Does it work?
It works!

### Double check that you've met specifications for each of the [rubric](https://review.udacity.com/#!/rubrics/1534/view) points.

# Extra Challenges: Real World Planning

For an extra challenge, consider implementing some of the techniques described in the "Real World Planning" lesson. You could try implementing a vehicle model to take dynamic constraints into account, or implement a replanning method to invoke if you get off course or encounter unexpected obstacles.